In [2]:
import os
import math

from functools import lru_cache
import pandas as pd
import numpy as np
from pypeg2 import *

import ucf101.filename_parser as filename_parser

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
root = '/home/will/thesis/generated/ucf101/test-1'
spatial_non_contrastive_pickle = os.path.join(root, 'spatial_non_contrastive_excitation_maps.pickle')
spatial_contrastive_pickle = os.path.join(root, 'spatial_contrastive_excitation_maps.pickle')
temporal_non_contrastive_pickle = os.path.join(root, 'temporal_non_contrastive_excitation_maps.pickle')
temporal_contrastive_pickle = os.path.join(root, 'temporal_contrastive_excitation_maps.pickle')

spatial_contrastive = pd.read_pickle(spatial_contrastive_pickle)
spatial_non_contrastive = pd.read_pickle(spatial_non_contrastive_pickle)
temporal_contrastive = pd.read_pickle(temporal_contrastive_pickle)
temporal_non_contrastive = pd.read_pickle(temporal_non_contrastive_pickle)


def clean_persisted_dataframe(df):
    """
    Instead of renaming all the columns of the data on disk and breaking all other code that uses it,
    we instead beautify it prior to using it for this analysis.
    """
    df.rename(columns={
        'attention_map': 'Attention Map',
        'frame': 'Frame',
        'starting_frame': 'Frame',
        'video': 'Clip',
        'type': 'Network',
    }, inplace=True)
    df['EBP Type'] = df['contrastive'].apply(lambda x: "Contrastive" if x else "Non-Contrastive")
    df.drop('contrastive', axis=1, inplace=True)
    capitalise = lambda s: s.title()
    df['Network'] = df['Network'].apply(capitalise)
    df['Window Size'] = df['Network'].apply(lambda net: 10 if net == "Temporal" else 1)
    return df
    
    
temporal_attention_maps = pd.concat([clean_persisted_dataframe(temporal_contrastive), 
                                     clean_persisted_dataframe(temporal_non_contrastive)])
spatial_attention_maps = pd.concat([clean_persisted_dataframe(spatial_contrastive), 
                                     clean_persisted_dataframe(spatial_non_contrastive)])
attention_maps = pd.concat([temporal_attention_maps, spatial_attention_maps])

def clip_video(clip):
    return compose(parse_clip(clip).video, autoblank=False)

def clip_action(clip):
    return compose(parse_clip(clip).action, autoblank=False)

@lru_cache()
def parse_clip(clip):
    return parse(clip, filename_parser.UCF101ActionClip)
    

clips = attention_maps['Clip'].unique()
clip_info = pd.DataFrame({
    'Clip': clips,
    'Video': list(map(clip_video, clips)),
    'Action': list(map(clip_action, clips)),
})
clip_info.set_index('Clip', inplace=True)

def get_clip_info(clip):
    return pd.Series([
        clip_info.loc[clip]['Video'],
        clip_info.loc[clip]['Action']
    ])
    

attention_maps[['Video', 'Action']] = attention_maps['Clip'].apply(get_clip_info)
attention_maps.head()

,Attention Map,Frame,Network,Clip,EBP Type,Window Size,Video,Action
0,"[[2.81474e-06, 3.89757e-06, 4.46529e-06, 6.075...",1,Temporal,v_BabyCrawling_g03_c04,Contrastive,10,03,BabyCrawling
1,"[[7.3387e-06, 8.75007e-06, 1.09497e-05, 1.3779...",2,Temporal,v_BabyCrawling_g03_c04,Contrastive,10,03,BabyCrawling
2,"[[2.18177e-05, 2.15572e-05, 2.30126e-05, 2.505...",3,Temporal,v_BabyCrawling_g03_c04,Contrastive,10,03,BabyCrawling
3,"[[4.21254e-06, 3.8012e-06, 2.36005e-06, 2.3341...",4,Temporal,v_BabyCrawling_g03_c04,Contrastive,10,03,BabyCrawling
4,"[[6.40428e-06, 5.55552e-06, 3.23084e-06, 3.134...",5,Temporal,v_BabyCrawling_g03_c04,Contrastive,10,03,BabyCrawling


In [4]:
attention_maps.to_pickle('attention-maps-ucf101-with-metadata.pickle')

In [6]:
clip_info.to_pickle('clip-info-ucf101.pickle')